In [1]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
import time 
import re 
import json 
import pandas as pd

In [2]:
url = "https://www.instagram.com/"
driver = wd.Chrome()
driver.get(url)

# 직접 로그인 하기
# 직접 검색어 입력하기

In [12]:
# 첫번째 게시물 
first_img_css="div.v1Nh3.kIKUG._bz0w" 
driver.find_element_by_css_selector(first_img_css).click()

In [7]:
# data lists 
location_infos = [] 
location_hrefs = [] 
upload_ids = [] 
date_texts = [] 
date_times = [] 
date_titles = [] 
main_texts = [] 
instagram_tags = [] 
comments = [] 
check_arrow = True 
count_extract = 0
# 크롤링 횟수
wish_num = 10
instagram_tags = [] 
instagram_tag_dates = []
imgsrclist = []

In [18]:
location_object_css="div.o-MQd.z8cbW > div.M30cS > div.JF9hh > a.O4GlU" 
upload_id_object_css="div.e1e1d > span.Jv7Aj.MqpiF > a.sqdOP.yWX7d._8A5w5.ZIAjV " 
date_object_css="div.k_Q0X.NnvRN > a.c-Yi7 > time._1o9PC.Nzb55" 
main_text_object_css="div.C7I1f.X7jCj > div.C4VMK > span" 
tag_css=".C7I1f.X7jCj" 
comment_more_btn="button.dCJp8.afkep" 
comment_ids_objects_css="ul.Mr508 > div.ZyFrc > li.gElp9.rUo9f > div.P9YgZ > div.C7I1f > div.C4VMK > h3" 
comment_texts_objects_css="ul.Mr508 > div.ZyFrc > li.gElp9.rUo9f > div.P9YgZ > div.C7I1f > div.C4VMK > span" 
print_flag=False 
next_arrow_btn_css1="._65Bje.coreSpriteRightPaginationArrow" 
next_arrow_btn_css2="._65Bje.coreSpriteRightPaginationArrow"

while True: 
    if count_extract > wish_num: 
        break 
    time.sleep(10) 
    
    # 위치정보 
    if check_arrow == False: 
        break 
    try: 
        location_object = driver.find_element_by_css_selector(location_object_css) 
        location_info = location_object.text 
        location_href = location_object.get_attribute("href") 
    except: 
        location_info = None 
        location_href = None 
    
    # 올린사람 ID 
    try: 
        upload_id_object = driver.find_element_by_css_selector(upload_id_object_css) 
        upload_id = upload_id_object.text 
    except: upload_id = None 
    
    # 날짜 
    try: 
        date_object = driver.find_element_by_css_selector(date_object_css) 
        date_text = date_object.text 
        date_time = date_object.get_attribute("datetime") 
        date_title = date_object.get_attribute("title") 
    except: 
        date_text = None 
        date_time = None 
        date_title = None 
    
    # 본문 
    try: 
        main_text_object = driver.find_element_by_css_selector(main_text_object_css) 
        main_text = main_text_object.text 
    except: 
        main_text = None 
    
    ## 본문 속 태그 
    try: 
        data = driver.find_element_by_css_selector(tag_css) # C7I1f X7jCj 
        tag_raw = data.text 
        tags = re.findall('#[A-Za-z0-9가-힣]+', tag_raw) 
        tag = ''.join(tags).replace("#"," ") # "#" 제거 
        tag_data = tag.split() 
        for tag_one in tag_data: 
            instagram_tags.append(tag_one) 
    except: 
        continue 
        
    # 이미지 src 주소 가져오기
    # div class="_97aPb " > img src = "~~"  
    try:
        image_src_element = driver.find_element_by_css_selector("div._97aPb img")
        image_src = image_src_element.get_attribute("src")
        imgsrclist.append(image_src)
        
    except:
        continue
    
    # 댓글 
    ## 더보기 버튼 클릭 
    
    try: 
        while True: 
            try: 
                more_btn = driver.find_element_by_css_selector(comment_more_btn) 
                more_btn.click() 
                time.sleep(1)
            except: 
                break 
    except: 
        print("----------------------fail to click more btn----------------------------------") 
        continue 
    
    ## 댓글 데이터 
    try: 
        comment_data = {} 
        comment_ids_objects = driver.find_elements_by_css_selector(comment_ids_objects_css) 
        comment_texts_objects = driver.find_elements_by_css_selector(comment_texts_objects_css) 
        try: 
            for i in range(len(comment_ids_objects)): 
                comment_data[str((i+1))] = {"comment_id":comment_ids_objects[i].text, "comment_text":comment_texts_objects[i].text} 
        except: 
            print("fail") 
    
    except: 
        comment_id = None 
        comment_text = None 
        comment_data = {} 
        
    try: 
        if comment_data != {}: 
            keys = list(comment_data.keys()) 
            
            for key in keys: 
                if comment_data[key]['comment_id'] == upload_id: 
                    tags = re.findall('#[A-Za-z0-9가-힣]+', comment_data[key]['comment_text']) 
                    tag = ''.join(tags).replace("#"," ") # "#" 제거 
                    
                    tag_data = tag.split() 
                    
                    for tag_one in tag_data: 
                        
                        instagram_tags.append(tag_one) 
    except: 
        continue 
        
    location_infos.append(location_info) 
    location_hrefs.append(location_href) 
    upload_ids.append(upload_id) 
    date_texts.append(date_text) 
    date_times.append(date_time) 
    date_titles.append(date_title) 
    main_texts.append(main_text) 
    comment_json = json.dumps(comment_data) 
    comments.append(comment_json) 
    
    if print_flag: 
        print("location_info : ", location_info) 
        print("location_href : ", location_href) 
        print("upload id : ", upload_id) 
        print("date : {} {} {}".format(date_text, date_time, date_title)) 
        print("main : ", main_text) 
        print("comment : ", comment_data) 
        print("insta tags : ", instagram_tags) 
    
    try: 
        WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CSS_SELECTOR, next_arrow_btn_css1))) 
        driver.find_element_by_css_selector(next_arrow_btn_css2).click() 
    
    except: 
        check_arrow = False 
    
    count_extract += 1

In [ ]:
save_file_name="instagram_extract" 
save_file_name_tag="instagram_tag" 
try: 
    insta_info_df = pd.DataFrame({"location_info":location_infos, "location_href":location_hrefs, "upload_id":upload_ids, "date_text":date_texts, "date_time":date_times, "date_title":date_titles, "main_text":main_texts, "comment":comments}) 
    insta_info_df.to_csv("{}.csv".format(save_file_name), index=False, encoding = "") 
except: 
    print("fail to save data") 

try: 
    insta_tag_df = pd.DataFrame({"tag":instagram_tags}) 
    insta_tag_df.to_csv("{}.csv".format(save_file_name_tag), index=False, encoding = "") 
    
except: 
    print("fail to save tag data")

In [ ]:
driver.close() 
driver.quit()